In [ ]:
#Install sagemaker_studio_image_build utility
import sys

!{sys.executable} -m pip install sagemaker_studio_image_build

In [19]:
import sagemaker
import boto3

try:
    role = sagemaker.get_execution_role()
except:
    role = get_execution_role()

print("Using IAM role arn: {}".format(role))

Using IAM role arn: arn:aws:iam::674880395054:role/service-role/AmazonSageMaker-ExecutionRole-20211101T135175


In [20]:
%%sh

sm-docker build . --repository bc-test-fcst:3.0

....[Container] 2023/01/06 19:02:00 going inside waitForAgent

[Container] 2023/01/06 19:02:00 Waiting for agent ping
[Container] 2023/01/06 19:02:01 Waiting for DOWNLOAD_SOURCE
[Container] 2023/01/06 19:02:03 Phase is DOWNLOAD_SOURCE
[Container] 2023/01/06 19:02:03 finished waitForAgent
[Container] 2023/01/06 19:02:03 CODEBUILD_SRC_DIR=/codebuild/output/src763069357/src
[Container] 2023/01/06 19:02:03 YAML location is /codebuild/output/src763069357/src/buildspec.yml
[Container] 2023/01/06 19:02:03 Setting HTTP client timeout to higher timeout for S3 source
[Container] 2023/01/06 19:02:03 Processing environment variables
[Container] 2023/01/06 19:02:04 No runtime version selected in buildspec.
[Container] 2023/01/06 19:02:04 Moving to directory /codebuild/output/src763069357/src
[Container] 2023/01/06 19:02:04 Configuring ssm agent with target id: codebuild:a7be3eea-14ea-413b-a26d-d05c33f29976
[Container] 2023/01/06 19:02:04 Successfully updated ssm agent configuration
[Container] 2023

In [21]:
from sagemaker.processing import ScriptProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role

role = get_execution_role()

processing_repo_uri = "674880395054.dkr.ecr.us-east-1.amazonaws.com/bc-test-fcst:3.0"

script_processor = ScriptProcessor(
    command=["python3"],
    image_uri=processing_repo_uri,
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
)

In [ ]:
# TEST predict.py
input_data = "/root/Blog_LegacyCode_MLOps/Template/inputdata/A207870_AI_ML_DEPLOYMENT_PREDICTIVE_EXPENCE_FORECASTING_DEMO.csv"
output_data = "/root/Blog_LegacyCode_MLOps/Template/outputdata/"
script_processor.run(
    code="src/predict.py",
    inputs=[ProcessingInput(source=input_data, destination="/opt/ml/processing/input")],
    outputs=[ProcessingOutput(output_name="pred", source="/opt/ml/processing/output",destination=output_data)]
)

In [31]:
import os
import json
import boto3
import time


sm = boto3.client('sagemaker')

# Get parameters
input_uri = "s3://sagemaker-us-east-1-674880395054/legacycode/data/input/test_input.csv"
ouput_uri = "s3://sagemaker-us-east-1-674880395054/legacycode/data/output/"
image_uri = "674880395054.dkr.ecr.us-east-1.amazonaws.com/bc-test-fcst:3.0"
role = get_execution_role()
instance_type = "ml.m5.xlarge"
volume_size = 20
max_runtime = 3600  # Default: 1h
entrypoint = "/opt/ml/code/predict.py"

timestamp = time.strftime('%Y%m%d-%H%M%S')
job_name = f'sm-job-{timestamp}' 

# Define inputs/outputs
inputs = [
    {
        'InputName': 'input_data',
        'S3Input': {
            'S3Uri': input_uri,
            'LocalPath': '/opt/ml/processing/input',
            'S3DataType': 'S3Prefix',
            'S3InputMode': 'File'
        }
    }
]

outputs = [
    {
        'OutputName': 'output_data',
        'S3Output': {
            'S3Uri': ouput_uri,
            'LocalPath': '/opt/ml/processing/output',
            'S3UploadMode': 'EndOfJob'
        }
    }
]

app_spec = {
    'ImageUri': image_uri,
    'ContainerEntrypoint': ['python', entrypoint]
}

cluster_config = {
    'InstanceCount': 1,
    'InstanceType': instance_type,
    'VolumeSizeInGB': volume_size
}

# Create processing job and return job ARN
sm.create_processing_job(
    ProcessingInputs=inputs,
    ProcessingOutputConfig={
        'Outputs': outputs
    },
    ProcessingJobName=job_name,
    ProcessingResources={
        'ClusterConfig': cluster_config
    },
    StoppingCondition={
        'MaxRuntimeInSeconds': max_runtime
    },
    AppSpecification=app_spec,
    RoleArn=role
)

{'ProcessingJobArn': 'arn:aws:sagemaker:us-east-1:674880395054:processing-job/sm-job-20230106-202951',
 'ResponseMetadata': {'RequestId': '0348e357-d574-48f0-b5f7-58cb5d21b5f8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0348e357-d574-48f0-b5f7-58cb5d21b5f8',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '101',
   'date': 'Fri, 06 Jan 2023 20:29:51 GMT'},
  'RetryAttempts': 0}}

In [33]:
job_name = 'sm-job-20230106-202951'

response = sm.describe_processing_job(
    ProcessingJobName=job_name
)
job_status = response["ProcessingJobStatus"]

print(job_status)

InProgress


In [35]:
from sagemaker import image_uris
image_uris.retrieve(framework='sklearn',region='us-east-1',version='0.23-1',image_scope='training')

'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3'

In [38]:
import os
import json
import boto3
import time


sm = boto3.client('sagemaker')

# Get parameters
input_uri = "s3://sagemaker-us-east-1-674880395054/legacycode/data/input/test_input.csv"
ouput_uri = "s3://sagemaker-us-east-1-674880395054/legacycode/data/output/"
code_uri = "s3://sagemaker-us-east-1-674880395054/legacycode/code/"
image_uri = "683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3"
role = get_execution_role()
instance_type = "ml.m5.xlarge"
volume_size = 20
max_runtime = 3600  # Default: 1h
entrypoint = "/opt/ml/input/data/custom_code/preprocess.py"

timestamp = time.strftime('%Y%m%d-%H%M%S')
job_name = f'sm-tr-job-{timestamp}' 

print("Training job", job_name)

create_training_params = {
    "AlgorithmSpecification": {"TrainingImage": image_uri, "TrainingInputMode": "File", "ContainerEntrypoint": ['python', entrypoint]},
    "RoleArn": role,
    "OutputDataConfig": {"S3OutputPath": ouput_uri},
    "ResourceConfig": {"InstanceCount": 1, "InstanceType": instance_type, "VolumeSizeInGB": 20},
    "TrainingJobName": job_name,
    "StoppingCondition": {"MaxRuntimeInSeconds": 60 * 60},
    "InputDataConfig": [
        {
            "ChannelName": "custom_code",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": code_uri,
                    "S3DataDistributionType": "FullyReplicated",
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None",
        },
        {
            'InputName': 'input_data',
            'S3Input': {
                'S3Uri': input_uri,
                'LocalPath': '/opt/ml/processing/input',
                'S3DataType': 'S3Prefix',
                'S3InputMode': 'File'
            }
        }
    ],
}


sagemaker = boto3.client("sagemaker")

sagemaker.create_training_job(**create_training_params)


Training job sm-tr-job-20230106-213126


{'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:674880395054:training-job/sm-tr-job-20230106-213126',
 'ResponseMetadata': {'RequestId': '0d487e1c-9383-4e14-93ad-df8583706bd3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0d487e1c-9383-4e14-93ad-df8583706bd3',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '100',
   'date': 'Fri, 06 Jan 2023 21:31:25 GMT'},
  'RetryAttempts': 0}}

In [41]:
job_name='sm-tr-job-20230106-213126'
status = sagemaker.describe_training_job(TrainingJobName=job_name)["TrainingJobStatus"]
print(status)

Failed


In [ ]:
import os
import json
import boto3
import time

from typing import Optional


sm = boto3.client('sagemaker')


def lambda_handler(event, context):

    # Get parameters
    input_uri = event['S3Input']
    ouput_uri = event['S3Output']
    image_uri = event['ImageUri']
    role = event['RoleArn']
    instance_type = event['InstanceType']
    volume_size = event['VolumeSizeInGB']
    max_runtime = event.get('MaxRuntimeInSeconds', 3600)  # Default: 1h
    container_arguments = event.get('ContainerArguments', None) # Optional
    entrypoint = event.get('EntryPoint', None) #optional 

    timestamp = time.strftime('%Y%m%d-%H%M%S')
    job_name = f'sagemaker-job-{timestamp}' 

    # Define inputs/outputs
    inputs = [
        'InputName': 'input_data',
        'S3Input': {
            'S3Uri': input_uri,
            'LocalPath': '/opt/ml/processing/input',
            'S3DataType': 'S3Prefix',
            'S3InputMode': 'File'
        }
    ]
    
    outputs = ['OutputName': 'output_data',
        'S3Output': {
            'S3Uri': ouput_uri,
            'LocalPath': '/opt/ml/processing/output',
            'S3UploadMode': 'EndOfJob'
        }
    ]
    
    # Define execution environment
    app_spec = {
        'ImageUri': image_uri
    }
    
    if container_arguments is not None:
        app_spec['ContainerArguments'] = container_arguments

    if entrypoint is not None:
        # Run a custom entrypoint within the container
        app_spec['ContainerEntrypoint'] = ['python', entrypoint]

    cluster_config = {
        'InstanceCount': 1,
        'InstanceType': instance_type,
        'VolumeSizeInGB': volume_size
    }

    # Create processing job and return job ARN
    sm.create_processing_job(
        ProcessingInputs=inputs,
        ProcessingOutputConfig={
            'Outputs': outputs
        },
        ProcessingJobName=job_name,
        ProcessingResources={
            'ClusterConfig': cluster_config
        },
        StoppingCondition={
            'MaxRuntimeInSeconds': max_runtime
        },
        AppSpecification=app_spec,
        RoleArn=role
    )

    return {
        'ProcessingJobName': job_name
    }

In [ ]:
import boto3

sm = boto3.client('sagemaker')

def lambda_handler(event, context):

    job_name = event['ProcessingJobName']

    response = sm.describe_processing_job(
        ProcessingJobName=job_name
    )
    job_status = response["ProcessingJobStatus"]
    
    print(response)
    
    return {
        'ProcessingJobStatus': job_status
    }

In [14]:
#Install requirements locally
!{sys.executable} -m pip install -r src/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 41.5 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
  Using cached setuptools_git-1.2-py2.py3-none-any.whl (10 kB)
  Using cached convertdate-2.4.0-py3-none-any.whl (47 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 kB 3.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 1.1 MB/s eta 0:00:00ta 0:00:01
  Using cached LunarCalendar-0.0.9-py2.py3-none-any.whl (18 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 37.8 MB/s eta 0:00:00:00:01
  Preparing metadata (setup.py) ... done
  Using cached hijri_converter-2.2.4-py3-none-any.whl (14 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.0 MB/s eta 0:00:00:00:01
  Created wheel for statistics: filename=statistics-1.0.3.5-py3-none-any.whl size=7451 sha256=2d6ce4540fba6100c7342ca0ae93aeca55483f8a8d6d6b16798c5c4456a4f3f9
  Stored in directory: /root/.cache/pip/wheels/e5/35/40/4a6e285553503f790eed

In [18]:
#Test locally
!python /root/Blog_LegacyCode_MLOps/Template/InferenceContainer/src/predict.py local /root/Blog_LegacyCode_MLOps/Template/data/

Started Run
Completed Run
